# Implementation of Homebrew Network

In [6]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))
from itertools import product

# Data imports
import cv2
import plotly.express as px
import numpy as np
import mlflow
import matplotlib.pyplot as plt

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer
from loss import CategoricalCrossEntropyLoss

## TESTING 
import importlib
importlib.reload(model)
##

<module 'model' from '..\\src\\model.py'>

## Data loader

In [7]:
def load_set(pth):
    X = []
    y = []

    for _class in os.listdir(pth):
        for img_name in os.listdir(os.path.join(pth, _class)):
            img = cv2.imread(os.path.join(pth, _class, img_name), cv2.IMREAD_GRAYSCALE)
            X.append(img)
            y.append(_class)

    X = np.array(X)

    # Normalizing 
    X = (X - np.mean(X))/ np.mean(X)

    # Vectorizing X
    X = X.reshape(X.shape[0], -1)

    # encoding y
    class_lbl_to_int = dict([(y, i) for i, y in enumerate(sorted(set(y)))])
    y = np.array([class_lbl_to_int[_y] for _y in y]).astype(np.uint8)
    y = one_hot_encode_index(y, len(class_lbl_to_int))

    # shuffling 
    keys = np.array(range(X.shape[0]))
    np.random.shuffle(keys)
    X = X[keys]
    y = y[keys]


    return X, y


def load_plant_data(resolution=32):
    base_dir = os.path.join(os.pardir, 'data', f'Plant_leave_diseases_{resolution}')
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')

    # loading train data 
    X_train, y_train = load_set(train_dir)

    # loading validation data 
    X_validation, y_validation = load_set(validation_dir)

    return X_train, y_train, X_validation, y_validation


## Setting up experiments

In [8]:
# Parimiters to test 
resolutions = [32]
leanring_rate = [0.001, 0.002]
decay = [1e-4, 1e-3]
dropout = [0.2, 0.9, 0.5]
configs = [{
    'resolution': cfg[0],
    'learning_rate': cfg[1],
    'decay': cfg[2],
    'dropout': cfg[3],
    'max_epochs': 200
    
} for cfg in product(resolutions, leanring_rate, decay, dropout)]

In [9]:
mlflow.set_experiment("Plant Leaf Disease")

X_train, y_train, X_validation, y_validation = load_plant_data()

for config in configs:
    
    # initialize model 
    mdl = model.Model(Adam(learning_rate=config['learning_rate'], decay=config['decay']),
                      CategoricalCrossEntropyLoss())

    # Config early stop 
    mdl.add_early_stop(25)

    # Defining architecture 
    mdl.set_sequence([
                        LinearLayer(X_train.shape[1], 1024),
                        ReLU(),
                        Dropout(config['dropout']),
                        LinearLayer(1024, 512), 
                        ReLU(),
                        Dropout(config['dropout']),
                        LinearLayer(512, 39),
                        Softmax()
                    ])


    with mlflow.start_run():
        mlflow.log_param('framework', 'homebrew')
        mlflow.log_param('data_split', '90/10')
        mlflow.log_param('type', 'FFNN')
        mlflow.log_params(config)
        mdl.train(X_train, y_train, epochs=config['max_epochs'], batch_size=64, validation=(X_validation, y_validation))
        

=== Epoch: 1 ===
Step: 0/866, accuracy0.000, loss3.687, learning rate 0.0010000 
Step: 100/866, accuracy0.156, loss3.328, learning rate 0.0009901 
Step: 200/866, accuracy0.172, loss2.996, learning rate 0.0009804 
Step: 300/866, accuracy0.156, loss2.985, learning rate 0.0009709 
Step: 400/866, accuracy0.219, loss2.878, learning rate 0.0009615 
Step: 500/866, accuracy0.281, loss2.512, learning rate 0.0009524 
Step: 600/866, accuracy0.219, loss2.595, learning rate 0.0009434 
Step: 700/866, accuracy0.312, loss2.718, learning rate 0.0009346 
Step: 800/866, accuracy0.312, loss2.546, learning rate 0.0009259 
Step: 865/866, accuracy0.302, loss2.535, learning rate 0.0009204 
Epoch: 1/200, accuracy0.222, loss2.930, learning rate 0.001
Estimated reamining runtime: 3:30:45.495319
--Validation--
Validation : Accuracy: 0.309, Loss: 8.039
=== Epoch: 2 ===
Step: 0/866, accuracy0.297, loss2.662, learning rate 0.0009203 
Step: 100/866, accuracy0.250, loss2.838, learning rate 0.0009119 
Step: 200/866, ac